# Retrieval-Augmented Generation: Question Answering based on Custom Dataset

Many use cases such as building a chatbot require text (text2text) generation models like **[BloomZ 7B1](https://huggingface.co/bigscience/bloomz-7b1)**, **[Flan T5 XXL](https://huggingface.co/google/flan-t5-xxl)**, and **[Flan T5 UL2](https://huggingface.co/google/flan-ul2)** to respond to user questions with insightful answers. The **BloomZ 7B1**, **Flan T5 XXL**, and **Flan T5 UL2** models have picked up a lot of general knowledge in training, but we often need to ingest and use a large library of more specific information.

In this notebook we will demonstrate how to use **AI21 Contextual Answers** to answer questions using a library of documents as a reference, by using document embeddings and retrieval. The embeddings are generated from **GPT-J-6B** embedding model. 

**This notebook serves a template such that you can easily replace the example dataset by your own to build a custom question and asnwering application.**

## Step 1. Deploy large language model (LLM) in SageMaker JumpStart

To better illustrate the idea, let's first deploy all the models that are required to perform the demo. You can choose either deploying all three Flan T5 XXL, BloomZ 7B1, and Flan UL2 models as the large language model (LLM) to compare their model performances, or select **subset** of the models based on your preference. To do that, you need modify the `_MODEL_CONFIG_` python dictionary defined as below.

In [22]:
!pip install --upgrade sagemaker --quiet
!pip install ipywidgets==7 --quiet

sagemaker

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
ipywidgets

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [23]:
import time
import sagemaker, boto3, json
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
import numpy as np
import requests

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
model_version = "*"

url = "https://31sqnogys6.execute-api.us-east-1.amazonaws.com/LATEST/HF"

In [24]:
def query_endpoint_with_json_payload(url, payload):
    response = requests.post(
        url,
        json=payload,
    )
    return response

def parse_response_model_ai21(query_response):
    model_predictions = query_response.json()
    generated_text = model_predictions["answer"]
    return generated_text

Please uncomment the entries as below if you want to deploy multiple LLM models to compare their performance.

In [25]:
_MODEL_CONFIG_ = {
    # pre-deploy via JS or API Gateway
    "AI21-Contextual-Answers" : {
        "url": url,
        "parse_function": parse_response_model_ai21,
    }
}

## Step 2. Ask a question to LLM without providing the context

To better illustrate why we need retrieval-augmented generation (RAG) based approach to solve the question and anwering problem. Let's directly ask the model a question and see how they respond.

In [54]:
sample_index = 1

sample = {
    0: {"question": "How to scale down SageMaker Asynchronous endpoint to zero?", 
        "context": """You can scale down the Amazon SageMaker Asynchronous Inference endpoint\
instance count to zero in order to save on costs when you are not actively processing\
requests. You need to define a scaling policy that scales on the "ApproximateBacklogPerInstance"\
custom metric and set the "MinCapacity" value to zero. For step-by-step instructions,\
please visit the `autoscale an asynchronous endpoint` section of the developer guide."""},
    
    1: {"question": "How can I be sure SageMaker protects my data security and privacy?",
        "context": """Amazon SageMaker does not use or share customer models, training data, or algorithms.\
We know that customers care deeply about privacy and data security. That's why AWS gives\
you ownership and control over your content through simple, powerful tools that allow you\
to determine where your content will be stored, secure your content in transit and at rest,\
and manage your access to AWS services and resources for your users. We also implement\
responsible and sophisticated technical and physical controls that are designed to \
prevent unauthorized access to or disclosure of your content. As a customer, you maintain\
ownership of your content, and you select which AWS services can process, store, \
and host your content. We do not access your content for any purpose without your consent
"""
       },
    
    2: {"question": "Which new GAI features will be launched on SageMaker at 2023?",
        "context": ""
    }
}

question, context = sample[sample_index].values()

In [27]:
payload = {
    "context": "",
    "question": question
}


for model_id in _MODEL_CONFIG_:
    query_response = query_endpoint_with_json_payload(
        url, payload
    )
    generated_texts = _MODEL_CONFIG_[model_id]["parse_function"](query_response)
    print(f"For model: {model_id}, the generated output is:\n{generated_texts}\n")

For model: AI21-Contextual-Answers, the generated output is:
Answer not in document



## Step 3. Improve the answer to the same question with insightful context


To better answer the question well, we provide extra contextual information, combine it with a prompt, and send it to model together with the question. Below is an example.

In [28]:
payload = {
    "context": context,
    "question": question
}

for model_id in _MODEL_CONFIG_:

    query_response = query_endpoint_with_json_payload(
        url, payload
    )
    generated_texts = _MODEL_CONFIG_[model_id]["parse_function"](query_response)
    print(
        f"For model: {model_id}, the generated output is:\n{generated_texts}"
    )


For model: AI21-Contextual-Answers, the generated output is:
In order to scale down the Amazon SageMaker Asynchronous Inference endpoint instance count to zero, you need to implement a scaling policy that scales on the "ApproximateBacklogPerInstance" custom metric and set the "MinCapacity" value to zero.


## Step 4. Use RAG based approach to identify the correct documents, and use them and question to query LLM


We plan to use document embeddings to fetch the most relevant documents in our document knowledge library and combine them with the prompt that we provide to LLM.

To achieve that, we will do following.

* **Generate embedings for each of document in the knowledge library with the GPT-J-6B embedding model.**
* **Identify top K most relevant documents based on user query.**
    * **For a query of your interest, generate the embedding of the query using the same embedding model.**
    * **Search the indexes of top K most relevant documents in the embedding space using the SageMaker KNN algorithm.**
    * **Use the indexes to retrieve the corresponded documents.**
* **Combine the retrieved documents with prompt and question and send them into LLM.**



Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt -- maximum sequence length of 1024 tokens. 

### 4.1 Deploying the model endpoint for GPT-J-6B embedding model

In this section, we will deploy the GPT-J-6B embedding model from the Jumpstart UI.


On the left-hand-side navigation pane, got to **Home**, under **SageMaker JumpStart**, choose **Model, notebooks, solutions**. You’re presented with a range of solutions, foundation models, and other artifacts that can help you get started with a specific model or a specific business problem or use case. If you want to experiment in a particular area, you can use the search function. Or you can simply browse the artifacts to find the relevant model or business solution for your needs. To start exploring the Stable Diffusion models, complete the following steps:

1. Go to the **Foundation Models** section. In the search bar, search for the **embedding** model and select the **GPT-J 6B Embedding FP16**.
<div>
    <img src="./img/embedding_model.png" alt="Image jumpstart" width="800" style="display:inline-block">
</div>
<br>

2. A new tab is opened with the options to train, deploy and view model details as shown below. In the Deploy Model section, expand Deployment Configuration. For SageMaker hosting instance, choose the hosting instance (for this lab, we use ml.g5.4xlarge). You can also change the Endpoint name as needed. Then click the Deploy button.

<div>
    <img src="./img/embedding_deploy.png" alt="Image deploy" width="600" style="display:inline-block">
</div>
<br>

3. The deploy action will start a new tab showing the model creation status and the model deployment status. Wait until the endpoint status shows **In Service**. This will take a few minutes.
<div>
    <img src="./img/ready.png" alt="Image ready" width="600" style="display:inline-block">
</div>
<br>

In [29]:
endpoint_name_embed = "jumpstart-dft-hf-textembedding-gpt-j-6b-fp16" # change the endpoint name as needed

In [30]:
from tqdm import tqdm
def query_endpoint_with_json_payload_endpoint(encoded_json, endpoint_name, content_type="application/json"):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType=content_type, Body=encoded_json
    )
    return response

def parse_response_multiple_texts(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    embeddings = model_predictions["embedding"]
    return embeddings


def build_embed_table(df_knowledge, endpoint_name_embed, col_name_4_embed, batch_size=10):
    res_embed = []
    N = df_knowledge.shape[0]
    for idx in tqdm(range(0, N, batch_size)):
        content = df_knowledge.loc[idx : (idx + batch_size - 1)][
            col_name_4_embed
        ].tolist()  ## minus -1 as pandas loc slicing is end-inclusive
        payload = {"text_inputs": content}
        query_response = query_endpoint_with_json_payload_endpoint(
            json.dumps(payload).encode("utf-8"), endpoint_name_embed
        )
        generated_embed = parse_response_multiple_texts(query_response)
        res_embed.extend(generated_embed)
    res_embed_df = pd.DataFrame(res_embed)
    return res_embed_df

### 4.2. Generate embedings for each of document in the knowledge library with the GPT-J-6B embedding model.

For the purpose of the demo we will use [Amazon SageMaker FAQs](https://aws.amazon.com/sagemaker/faqs/) as knowledge library. The data are formatted in a CSV file with two columns Question and Answer. We use **only** the Answer column as the documents of knowledge library, from which relevant documents are retrieved based on a query. 

**Each row in the CSV format dataset corresponds to a textual document. 
We will iterate each document to get its embedding vector via the GPT-J-6B embedding models. 
For your purpose, you can replace the example dataset of your own to build a custom question and answering application.**


First, we download the dataset from our S3 bucket to the local.

In [31]:
s3_path = f"s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv"

In [64]:
# Downloading the Database
!aws s3 cp $s3_path Amazon_SageMaker_FAQs.csv
!cat ./Amazon_SageMaker_FAQs.csv

download: s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv to ./Amazon_SageMaker_FAQs.csv
﻿What is Amazon SageMaker?,"Amazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine learning (ML) models for any use case with fully managed infrastructure, tools, and workflows."
"In which Regions is Amazon SageMaker available?
","For a list of the supported Amazon SageMaker AWS Regions, please visit the AWS Regional Services page. Also, for more information, see Regional endpoints in the AWS general reference guide."
"What is the service availability of Amazon SageMaker?
","Amazon SageMaker is designed for high availability. There are no maintenance windows or scheduled downtimes. SageMaker APIs run in Amazon’s proven, high-availability data centers, with service stack replication configured across three facilities in each AWS Region to provide fault tolerance in the event of a server failure or Availab

In [38]:
import pandas as pd

df_knowledge = pd.read_csv("Amazon_SageMaker_FAQs.csv", header=None, usecols=[1], names=["Answer"])
df_knowledge.head(6)

,Answer
0,Amazon SageMaker is a fully managed service to...
1,For a list of the supported Amazon SageMaker A...
2,Amazon SageMaker is designed for high availabi...
3,Amazon SageMaker stores code in ML storage vol...
4,Amazon SageMaker ensures that ML model artifac...
5,Amazon SageMaker does not use or share custome...


In [39]:
df_knowledge_embed = build_embed_table(
    df_knowledge, endpoint_name_embed, col_name_4_embed='Answer', batch_size=20
)
df_knowledge_embed.head(5)

100%|██████████| 8/8 [00:10<00:00,  1.28s/it]


,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,-0.004362,-0.004465,-0.012076,-0.020756,-0.032897,0.020553,-0.011508,0.031901,0.007894,0.033841,...,-0.005610,-0.004192,0.015780,-0.004793,0.013671,0.012543,0.020478,-0.003520,-0.016619,-0.014112
1,0.020732,0.001020,-0.001266,0.007799,-0.017973,0.028117,-0.008295,0.025874,0.024099,0.006966,...,-0.006944,0.020163,0.000506,-0.005667,0.026802,0.009597,0.022600,0.011659,-0.025841,-0.019778
2,0.000593,0.007449,-0.010419,-0.009703,-0.002267,0.029763,-0.003595,0.015971,0.016034,0.004429,...,-0.001741,0.003375,0.008886,-0.009860,0.016276,0.008298,0.034845,-0.012401,-0.011383,-0.011228
3,0.005865,0.001990,-0.010981,0.002104,0.003247,0.021572,0.004312,0.024008,0.028580,0.003881,...,-0.010760,0.007404,-0.001815,0.010746,0.022047,0.003767,0.020633,-0.014895,-0.013624,-0.021510
4,0.004386,0.000713,-0.006952,0.005698,-0.019112,0.015790,0.003119,0.023976,0.017843,0.004850,...,-0.025475,0.007318,0.013796,0.009254,0.026327,0.011818,0.016824,-0.001514,-0.016030,-0.017670


In [40]:
assert df_knowledge_embed.shape[0] == df_knowledge.shape[0]

Save the embedding data for further usage.

In [68]:
df_knowledge_embed.to_csv("Amazon_SageMaker_FAQs_embedding.csv", header=None, index=False)

### 4.3. Index the embedding knowledge library using the [KNN algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/k-nearest-neighbors.html)

You can choose to use the SageMaker KNN, which will conduct following.

1. Start a training job to index the embedding knowledge data. The underlying algorithm used to index the data is [Faiss](https://github.com/facebookresearch/faiss).
2. Start an endpoint to take the embedding of the query as input and return the top K nearest indexes of the documents.

**Note.** For the KNN training job, the features are N by P matrix, where N is the number of documetns in the knowledge library, P is the embedding dimension, and each row corresponds to an embedding of a document. The labels are ordinal integers starting from 0. During inference, given an embedding of query, the labels of the top K nearest documents with respect to the query are used as indexes to retrieve the corresponded textual documents.

In this example, considering the data we are using is quite small. We will train a KNN model using [SKLearn] (https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) and perform local invocations in the notebook.




In [69]:
import numpy as np
import os
import io


train_features = np.array(df_knowledge_embed)

# Providing each answer embedding label
train_labels = np.array([i for i in range(len(train_features))])

print("train_features shape = ", train_features.shape)
print("train_labels shape = ", train_labels.shape)


train_features shape =  (154, 4096)
train_labels shape =  (154,)


We want to retrieve the top 5 most relevant documents. 

In [70]:
TOP_K = 5

The below code train a KNN model locallly.

In [71]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=TOP_K, metric="cosine")
neigh.fit(train_features)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

### 4.4 Retrieve the most relevant documents

Given the embedding of a query, we will query the endpoint to get the indexes of top K most relevant documents and use the indexes to retrieve the corresponded textual documents.

Next, the textual documents are concatenated with maximum length of `MAX_SECTION_LEN`. This is to make sure the context we send into the prompt contains a good enough amount of information all the while not exceeding model's capacity.

In [91]:
MAX_SECTION_LEN = 1500
THRESHOLD = 0.2 # lower --> less permissive (need to be closer)
SEPARATOR = "\n* "


def construct_context(
    context_predictions_idx, 
    context_prediction_dist, 
    df_knowledge, 
    threshold, 
    context_length
) -> str:
    chosen_sections = []
    chosen_sections_len = 0

    for index, dist in zip(context_predictions_idx, context_prediction_dist):
        document_section = df_knowledge.loc[index]
        chosen_sections_len += len(document_section) + 2
        if dist > threshold:
            break

        chosen_sections.append(SEPARATOR + document_section.replace("\n", " "))
    concatenated_doc = "".join(chosen_sections)[:context_length]
    print(
        f"With maximum sequence length {MAX_SECTION_LEN}, \
selected top {len(chosen_sections)} document sections: {concatenated_doc}"
    )

    return concatenated_doc, len(chosen_sections)

## Convert Question to Embedding

In [92]:
question = 'Which hardware platforms does Amazon SageMaker Neo support?'

query_response = query_endpoint_with_json_payload_endpoint(
    question, endpoint_name_embed, content_type="application/x-text"
)
question_embedding = parse_response_multiple_texts(query_response)
print(question, np.array(question_embedding), sep="\n")

Which hardware platforms does Amazon SageMaker Neo support?
[[-0.00091581 -0.00500719 -0.00332222 ... -0.02235409 -0.00776587
  -0.00274346]]


## Retrive the Most Relevant Context

In [93]:
# Getting the most relevant context using KNN
distances, indices = neigh.kneighbors(question_embedding)
distances, indices

(array([[0.24066303, 0.24921681, 0.25470706, 0.26667646, 0.26688052]]),
 array([[134,  16, 124,  91, 152]]))

In [95]:
print(question)
context_retrieve, num_context_doc = construct_context(
    indices[0], 
    distances[0], 
    df_knowledge["Answer"], 
    threshold=0.3,
    context_length=MAX_SECTION_LEN
)

Which hardware platforms does Amazon SageMaker Neo support?
With maximum sequence length 1500, selected top 5 document sections: 
* Amazon SageMaker Edge Manager supports common CPU (ARM, x86) and GPU (ARM, Nvidia) based devices with Linux and Windows operating systems. Over time, SageMaker Edge Manager will expand to support more embedded processors and mobile platforms that are also supported by SageMaker Neo.
* You can find the Regions where Amazon SageMaker Studio is supported in the documentation here.
* Yes, we support all types of containers. Amazon EC2 Inf1, based on the AWS Inferentia chip, requires a compiled model artifact using either the Neuron compiler or Amazon SageMaker Neo. Once you have a compiled model for an Inferentia target and the associated container image URI, you can use Amazon SageMaker Inference Recommender to benchmark different Inferentia instance types.
  Managed Spot Training is supported in all AWS Regions where Amazon SageMaker is currently available.


## Combine the retrieved documents, prompt, and question to query the LLM

In [101]:
print(question)

for model_id in _MODEL_CONFIG_:
    
    if num_context_doc == 0:
        print(f"Sorry, I don't have information to answer this question:\n{question}.")
        break
    
    payload = {"context": context_retrieve, "question": question}
    print(f'payload: {json.dumps(payload)}')
    
    query_response = query_endpoint_with_json_payload(
        url, payload
    )
    generated_texts = _MODEL_CONFIG_[model_id]["parse_function"](query_response)
    print(f"\nAnswer from model: {model_id}:\n{generated_texts}\n")

Which hardware platforms does Amazon SageMaker Neo support?
payload: {"context": "\n* Amazon SageMaker Edge Manager supports common CPU (ARM, x86) and GPU (ARM, Nvidia) based devices with Linux and Windows operating systems. Over time, SageMaker Edge Manager will expand to support more embedded processors and mobile platforms that are also supported by SageMaker Neo.\n* You can find the Regions where Amazon SageMaker Studio is supported in the\u00a0documentation here.\n* Yes, we support all types of containers. Amazon EC2 Inf1, based on the AWS Inferentia chip, requires a compiled model artifact using either the Neuron compiler or Amazon\u00a0SageMaker Neo. Once you have a compiled model for an Inferentia target and the associated container image URI, you can use Amazon\u00a0SageMaker Inference Recommender to benchmark different Inferentia instance types.\n* Managed Spot Training is supported in all AWS Regions where Amazon SageMaker is currently available.\r \n* The difference between

### 4.6 Clean up

Uncomment below cell to delete the endpoint after testing. 

In [ ]:
# # delete the endpoints hosting the embedding model
# sagemaker_session.delete_endpoint(endpoint_name_embed)

## Bonus

### Interact with the model

**AI21 Studio Contextual Answers model** allows you to access our high-quality question answering technology. It was designed to answer questions based on a specific document context provided by the customer. This avoids any factual issues that language models may have and makes sure the answers it provides are grounded in that context document.

This model receives document text, serving as a context, and a question and returns an answer based entirely on this context. This means that if the answer to your question is not in the document, the model will indicate it (instead of providing a false answer).

To get a sense of the model's behavior, let's use this toy example of asking what is the Eiffel tower height. Most language models will simply answer according to their training data.

This model, however, bases its answer solely on the context you provide. Let's use the following [Wikipedia paragraph](https://en.wikipedia.org/wiki/Eiffel_Tower#:~:text=The%20Eiffel%20Tower%20(%2F%CB%88a%C9%AA,from%20the%20Champ%20de%20Mars) as context, with small modifications:

In [97]:
# Actual paragraph
context = "The tower is 330 metres (1,083 ft) tall,[6] about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest human-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure in the world to surpass both the 200-metre and 300-metre mark in height. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

# The paragraph with manual changes of the height
false_context = "The tower is 3 metres (10 ft) tall,[6] about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest human-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure in the world to surpass both the 200-metre and 300-metre mark in height. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

# The paragraph with the height omitted
partial_context = "Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest human-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure in the world to surpass both the 200-metre and 300-metre mark in height. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

In [98]:
# True context
payload = {
    "context": context,
    "question": "What is the height of the Eiffel tower?"
}
query_endpoint_with_json_payload(url, payload).json()

{'answer': 'The Eiffel Tower is 330 metres tall.'}

In [ ]:
# True context
payload = {
    "context": false_context,
    "question": "What is the height of the Eiffel tower?"
}
query_endpoint_with_json_payload(url, payload).json()

In [ ]:
# True context
payload = {
    "context": partial_context,
    "question": "What is the height of the Eiffel tower?"
}
query_endpoint_with_json_payload(url, payload).json()

### Flan-t5-xxl: Hallucination

In [ ]:
def parse_response_model_flan_t5(query_response):
    model_predictions = json.loads(json.dumps(query_response.json()))
    generated_text = model_predictions["generated_texts"]
    return generated_text

flan_t5_xxl_url = "https://yyosweaa3e.execute-api.us-east-1.amazonaws.com/LATEST/HF"

In [ ]:
payload = {
    "text_inputs": f"""Answer based on context:\n\n{partial_context}\n\nWhat is the height of the Eiffel tower?""",
    "max_length": 200,
    "num_return_sequences": 1,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": False,
    "temperature": .0,
}

print(payload["text_inputs"])


query_response = query_endpoint_with_json_payload(
    flan_t5_xxl_url, payload
)
generated_texts = parse_response_model_flan_t5(query_response)
print(f"For model: Flan-t5-xxl, the generated output is:\n{generated_texts[0]}\n")

### Ask about financial reports

The document context should be **no more than 10,000 characters**, and the question can be up to 160 characters.

Imagine you are performing research and rely on financial reports to base your findings. Let's take the following part from [JPMorgan Chase & Co. 2021 annual report](https://www.jpmorganchase.com/content/dam/jpmc/jpmorgan-chase-and-co/investor-relations/documents/annualreport-2021.pdf):

In [ ]:
financial_context = """In 2020 and 2021, enormous QE — approximately $4.4 trillion, or 18%, of 2021 gross domestic product (GDP) — and enormous fiscal stimulus (which has been and always will be inflationary) — approximately $5 trillion, or 21%, of 2021 GDP — stabilized markets and allowed companies to raise enormous amounts of capital. In addition, this infusion of capital saved many small businesses and put more than $2.5 trillion in the hands of consumers and almost $1 trillion into state and local coffers. These actions led to a rapid decline in unemployment, dropping from 15% to under 4% in 20 months — the magnitude and speed of which were both unprecedented. Additionally, the economy grew 7% in 2021 despite the arrival of the Delta and Omicron variants and the global supply chain shortages, which were largely fueled by the dramatic upswing in consumer spending and the shift in that spend from services to goods. Fortunately, during these two years, vaccines for COVID-19 were also rapidly developed and distributed.
In today's economy, the consumer is in excellent financial shape (on average), with leverage among the lowest on record, excellent mortgage underwriting (even though we've had home price appreciation), plentiful jobs with wage increases and more than $2 trillion in excess savings, mostly due to government stimulus. Most consumers and companies (and states) are still flush with the money generated in 2020 and 2021, with consumer spending over the last several months 12% above pre-COVID-19 levels. (But we must recognize that the account balances in lower-income households, smaller to begin with, are going down faster and that income for those households is not keeping pace with rising inflation.)
Today's economic landscape is completely different from the 2008 financial crisis when the consumer was extraordinarily overleveraged, as was the financial system as a whole — from banks and investment banks to shadow banks, hedge funds, private equity, Fannie Mae and many other entities. In addition, home price appreciation, fed by bad underwriting and leverage in the mortgage system, led to excessive speculation, which was missed by virtually everyone — eventually leading to nearly $1 trillion in actual losses.
"""

Rather than reading the entire report, just ask what you want to know. The model will answer based on the provided report:

In [ ]:
payload = {
    "context": financial_context,
    "question": "Did the economy shrink after the Omicron variant arrived?"
}
query_endpoint_with_json_payload(url, payload).json()

In addition, you can ask more complex questions, where the answer requires deductions rather than just extracting the correct sentence from the document context. This will result in abstractive, rather than extractive, answers that draw on several different parts of the document. For example, look at the following question:

In [ ]:
payload = {
    "context": financial_context,
    "question":  "Did COVID-19 eventually help the economy?"
}
query_endpoint_with_json_payload(url, payload).json()

We now present the model with the following question. You may be confused to answer something based on the last paragraph without delving into the text. However, if you read the provided document context properly, you will discover that the answer does not appear there. The model handles this as expected:

In [ ]:
payload = {
    "context": financial_context,
    "question":  "How did COVID-19 affect the financial crisis of 2008?"
}
query_endpoint_with_json_payload(url, payload).json()